In [ ]:
import numpy as np
import pandas as pd
import os
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from sklearn.preprocessing import minmax_scale
import IPython.display as ipd

plt.rcParams['figure.figsize'] = (20,8)
plt.rcParams['font.size'] = 16
sns.set_style('darkgrid')
warnings.filterwarnings("ignore")

# Emotion Detection
- It is a system through which various audio speech files are classified into different emotions such as happy, sad, anger and neutral by computers. Speech emotion recognition can be used in areas such as the medical field or customer call centers.
- In this kernel the emotions will be classified into only 2 categories: happy or sad

<img src='https://miro.medium.com/max/1185/1*6erovyvVqpraE2VR0f3vfw.jpeg'>

<div class='alert alert-info'><strong>Note: </strong>Here the emotion detection is being performed on the acoustic features of the data and not the semantic understanding of the sentences spoken by a person</div>

# Data Preparation
- Create a dataframe containing the meta data
- Create a list of all combinations given in the specification
- Filter out the data using these combinations
- Assign classes to each record (happy or sad)
- Assign gender class to each record so that it can be leveraged during analysis

In [ ]:
records = []
for dirname, _, filenames in os.walk('ravdess_data'):
    for filename in filenames:
        records.append([filename, os.path.join(dirname,filename)])

data = pd.DataFrame(records, columns=['filename','path'])
data['actor'] = data['path'].apply(lambda x: re.findall("\w+_\d+",x)[0])
data = data[data['actor']!="audio_speech_actors_01"]
data.reset_index(inplace=True,drop=True)
data['type'] = data['filename'].apply(lambda x: re.split("-\d+\.wav",x)[0])
data

## Number of Unique Speakers/Actors

In [ ]:
data['actor'].nunique()

## Number of Audio files for each of the Speakers/Actors

In [ ]:
data['actor'].value_counts()

In [ ]:
filter_list = []
for a in ["03","04"]:
    for b in ["01","02"]:
        for c in ["01","02"]:
            for d in ["01","02"]:
                filter_list.append(f"03-01-{a}-{b}-{c}-{d}")
                
data = data[data['type'].isin(filter_list)]
data.reset_index(inplace=True,drop=True)
data['class'] = data['type'].apply(lambda x: 'happy' if x.startswith('03-01-03') else 'sad')
data['gender'] = data['actor'].apply(lambda x: 'female' if int(x.split('_')[1])%2==0 else 'male')

In [ ]:
data

## Number of Audio Files for Gender and Emotion Class

In [ ]:
data['gender'].value_counts()

In [ ]:
data['class'].value_counts()

# Feature Extraction 
- The acoustic features of an audio can be extracted using different methods, but in this kernel only 2 methods will be used
1. MFCCs
2. Mel Spectrograms

<img src='https://images.deepai.org/converted-papers/2005.12779/x3.png'>

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def feature_extraction(df, mfcc=True):
    features = []
    for i,record in tqdm(df.iterrows(),total=df.shape[0]):
        x , sr = librosa.load(record['path'])
        mean_mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=128),axis=1)
        mean_ms = np.mean(librosa.feature.melspectrogram(y=x, sr=sr, n_mels=128),axis=1)
        features.append(mean_mfcc if mfcc else mean_ms)
        
    dataf = pd.DataFrame(features)
    dataf['class'] = df['class']
    return dataf

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

## MFCC Features

In [ ]:
dataf = feature_extraction(data)
dataf

In [ ]:
X1 = dataf.iloc[:,:-1].values
y1 = dataf.iloc[:,-1].values 
y1 = encoder.fit_transform(y1)

## Mel Spectrogram Features

In [ ]:
dataf = feature_extraction(data,mfcc=False)
dataf

In [ ]:
X2 = dataf.iloc[:,:-1].values
y2 = dataf.iloc[:,-1].values
y2 = encoder.fit_transform(y2)

# Modelling
- The following models will be used for building the models for emotion detection
1. Logistic Regression on MFCC and Mel Spec Features
2. CNNs on MFCC and Mel Spec Features
3. Autoencoders and Variational Autoencoders on whichever features are giving better performance in the above models

## Logistic Regression

<img src='https://miro.medium.com/max/1400/1*Ba7LqnrsRnhjJyJl5LPW6Q.gif'>

In [ ]:
def LogisticRegressionPipeline(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    pipeline = Pipeline([('scaler',StandardScaler()),('LogisticRegression',LogisticRegression())])
    pipeline.fit(X_train,y_train)
    y_train_pred = pipeline.predict(X_train)
    y_pred = pipeline.predict(X_test)
    
    cmatrix = confusion_matrix(y_test,y_pred)
    
    print("Training Performance")
    print(classification_report(y_train,y_train_pred))
    print("-----------------------------------------")
    print("Test Performance")
    print(classification_report(y_test,y_pred))
    print("-----------------------------------------")
    
    cv_score = cross_val_score(pipeline,X,y,cv=5)
    average = lambda x: sum(x)/len(x)
    print("5-Folds Scores: ", cv_score)
    print("-----------------------------------------")
    print("5-Folds Average Score: ",average(cv_score))
    
    return cv_score, cmatrix

## Logistic Regression: MFCCs

In [ ]:
scores, cmatrix = LogisticRegressionPipeline(X1,y1)

In [ ]:
sns.heatmap(cmatrix,annot=True,fmt='.3g',cmap='viridis',xticklabels=['happy','sad'],yticklabels=['happy','sad'])
plt.show()

## Logistic Regression: Mel Spectrogram

In [ ]:
scores, cmatrix = LogisticRegressionPipeline(X2,y2)

In [ ]:
sns.heatmap(cmatrix,annot=True,fmt='.3g',cmap='viridis',xticklabels=['happy','sad'],yticklabels=['happy','sad'])
plt.show()

## CNNs (Convolutional Neural Networks)

<img src='https://miro.medium.com/max/1400/1*ciDgQEjViWLnCbmX-EeSrA.gif'>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, InputLayer, UpSampling2D, Layer, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## CNNs: MFCCs

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, stratify=y1)
X_train = X_train.reshape(-1,16,8,1)
X_test = X_test.reshape(-1,16,8,1)

In [ ]:
model = Sequential([
    InputLayer(input_shape=(16, 8, 1)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = "same"),
    MaxPooling2D(2, 2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = "same"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_name = "model.h5"
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test,y_test), callbacks=[checkpoint,earlystopping])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0

In [ ]:
print(classification_report(y_test,y_pred))
cmatrix = confusion_matrix(y_test,y_pred)
sns.heatmap(cmatrix,annot=True,fmt='.3g',cmap='viridis',xticklabels=['happy','sad'],yticklabels=['happy','sad'])
plt.show()

## CNNs: Mel Spectrogram

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, stratify=y2)
X_train = X_train.reshape(-1,16,8,1)
X_test = X_test.reshape(-1,16,8,1)

In [ ]:
model = Sequential([
    InputLayer(input_shape=(16, 8, 1)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = "same"),
    MaxPooling2D(2, 2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = "same"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_name = "model_mel.h5"
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test,y_test), callbacks=[checkpoint,earlystopping])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim([0,2])
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))
cmatrix = confusion_matrix(y_test,y_pred)
sns.heatmap(cmatrix,annot=True,fmt='.3g',cmap='viridis',xticklabels=['happy','sad'],yticklabels=['happy','sad'])
plt.show()

**Inference:** MFCC Features give better performance

## Autoencoders
<img src='https://miro.medium.com/max/1400/1*44eDEuZBEsmG_TCAKRI3Kw@2x.png'>

In [ ]:
dataf = feature_extraction(data)
dataf

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [ ]:
X_happy = dataf[dataf['class']=='happy'].iloc[:,:-1].values 
X_sad = dataf[dataf['class']=='sad'].iloc[:,:-1].values 

X_happy = scaler.fit_transform(X_happy)
X_sad = scaler.transform(X_sad)

X_happy = X_happy.reshape(-1,16,8,1)
X_sad = X_sad.reshape(-1,16,8,1)

In [ ]:
model = Sequential([
    Input(shape=(16,8,1)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    UpSampling2D((2,2)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    UpSampling2D((2,2)),
    Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same')
])

model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_happy,X_happy,epochs=100)

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.evaluate(X_happy,X_happy)

In [ ]:
model.evaluate(X_sad,X_sad)

**Inference:** There is not a significant difference between the loss for happy audios and sad audios

## Variational Autoencoders
<img src='https://www.jeremyjordan.me/content/images/2018/03/Screen-Shot-2018-03-18-at-12.24.19-AM.png'>

In [ ]:
class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
input_layer = Input(shape=(16,8,1))
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D(2,2)(x)
x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D(2,2)(x)
x = Flatten()(x)
z_mean = Dense(128, name="z_mean")(x)
z_log_var = Dense(128, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
x = Dense(4*2*64, activation="relu")(z)
x = Reshape((4, 2, 64))(x)
x = UpSampling2D()(x)
x = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(x)
x = UpSampling2D()(x)
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(x)
output = Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same')(x)

model = Model(inputs=input_layer,outputs=output)
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

In [ ]:
history = model.fit(X_happy,X_happy,epochs=100)

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.evaluate(X_happy,X_happy)

In [ ]:
model.evaluate(X_sad,X_sad)

**Inference:** There is a significant difference between the loss of happy and sad audios

In [ ]:
X_sc = np.vstack([X_happy,X_sad])
y = np.hstack([np.ones((192,)),np.zeros((192,))])

In [ ]:
var_encoder = Model(inputs=model.input,outputs=model.layers[6].output)
var_encoder.summary()

In [ ]:
z_mean = var_encoder.predict(X_sc)

In [ ]:
sns.scatterplot(z_mean[:, 0], z_mean[:, 1], hue=y, s=80)
plt.title("Variational Autoencoder Latent Space Representation in 2D")
plt.xlabel("z[0]")
plt.ylabel("z[1]")
plt.show()

<div class='alert alert-success'><strong>Conclusion:</strong>
    <li><strong>Logistic Regression</strong> model performs well with the MFCC Features with a decent classifcation accuracy and F1-score</li>
    <li><strong>CNNs</strong> perform well on the MFCC Features as well, therefore for autoencoders only MFCCs are taken as features</li>
    <li><strong>Variational Autoencoders</strong> perform a better job in terms of reconstruction loss than the regular autoencoders as they are able to model the distribution of happy audios in <strong>128-Dimensional latent space</strong></li>
</div>